In [1]:
import sys
sys.path.append("..")

In [2]:
from collections import Counter, defaultdict
from time import time

from resources.utils import get_puzzle_input

### Part 1

The Elves play this game by taking turns arranging the marbles in a circle according to very particular rules. The marbles are numbered starting with 0 and increasing by 1 until every marble has a number.

First, the marble numbered 0 is placed in the circle. At this point, while it contains only a single marble, it is still a circle: the marble is both clockwise from itself and counter-clockwise from itself. This marble is designated the current marble.

Then, each Elf takes a turn placing the lowest-numbered remaining marble into the circle between the marbles that are 1 and 2 marbles clockwise of the current marble. (When the circle is large enough, this means that there is one marble between the marble that was just placed and the current marble.) The marble that was just placed then becomes the current marble.

However, if the marble that is about to be placed has a number which is a multiple of 23, something entirely different happens. First, the current player keeps the marble they would have placed, adding it to their score. In addition, the marble 7 marbles counter-clockwise from the current marble is removed from the circle and also added to the current player's score. The marble located immediately clockwise of the marble that was removed becomes the new current marble.

For example, suppose there are 9 players. After the marble with value 0 is placed in the middle, each player (shown in square brackets) takes a turn. The result of each of those turns would produce circles of marbles like this, where clockwise is to the right and the resulting current marble is in parentheses:

```
[-] (0)
[1]  0 (1)
[2]  0 (2) 1 
[3]  0  2  1 (3)
[4]  0 (4) 2  1  3 
[5]  0  4  2 (5) 1  3 
[6]  0  4  2  5  1 (6) 3 
[7]  0  4  2  5  1  6  3 (7)
[8]  0 (8) 4  2  5  1  6  3  7 
[9]  0  8  4 (9) 2  5  1  6  3  7 
[1]  0  8  4  9  2(10) 5  1  6  3  7 
[2]  0  8  4  9  2 10  5(11) 1  6  3  7 
[3]  0  8  4  9  2 10  5 11  1(12) 6  3  7 
[4]  0  8  4  9  2 10  5 11  1 12  6(13) 3  7 
[5]  0  8  4  9  2 10  5 11  1 12  6 13  3(14) 7 
[6]  0  8  4  9  2 10  5 11  1 12  6 13  3 14  7(15)
[7]  0(16) 8  4  9  2 10  5 11  1 12  6 13  3 14  7 15 
[8]  0 16  8(17) 4  9  2 10  5 11  1 12  6 13  3 14  7 15 
[9]  0 16  8 17  4(18) 9  2 10  5 11  1 12  6 13  3 14  7 15 
[1]  0 16  8 17  4 18  9(19) 2 10  5 11  1 12  6 13  3 14  7 15 
[2]  0 16  8 17  4 18  9 19  2(20)10  5 11  1 12  6 13  3 14  7 15 
[3]  0 16  8 17  4 18  9 19  2 20 10(21) 5 11  1 12  6 13  3 14  7 15 
[4]  0 16  8 17  4 18  9 19  2 20 10 21  5(22)11  1 12  6 13  3 14  7 15 
[5]  0 16  8 17  4 18(19) 2 20 10 21  5 22 11  1 12  6 13  3 14  7 15 
[6]  0 16  8 17  4 18 19  2(24)20 10 21  5 22 11  1 12  6 13  3 14  7 15 
[7]  0 16  8 17  4 18 19  2 24 20(25)10 21  5 22 11  1 12  6 13  3 14  7 15
```

The goal is to be the player with the highest score after the last marble is used up. Assuming the example above ends after the marble numbered 25, the winning score is 23+9=32 (because player 5 kept marble 23 and removed marble 9, while no other player got any points in this very short example game).

Here are a few more examples:

```
10 players; last marble is worth 1618 points: high score is 8317
13 players; last marble is worth 7999 points: high score is 146373
17 players; last marble is worth 1104 points: high score is 2764
21 players; last marble is worth 6111 points: high score is 54718
30 players; last marble is worth 5807 points: high score is 37305
```

What is the winning Elf's score?

In [3]:
class CircularBuffer:
    def __init__(self, values, position=0):
        self.buffer = values
        self.position = position
        
    def __len__(self):
        return len(self.buffer)
    
    def move(self, offset=0):
        position = self._position_from_offset(offset)
        self.position = position
        return position

    def value(self, offset=0):
        position = self._position_from_offset(offset)
        return self.buffer[position]
    
    def insert(self, value, offset=0):
        position = self._position_from_offset(offset)
        self.buffer.insert(position, value)
        
    def remove(self, offset=0):
        remove_position = self._position_from_offset(offset)

        if remove_position <= self.position:
            self.position -= 1
        value = self.buffer[remove_position]
        del self.buffer[remove_position]
        return value

    def _move_op(self, offset):
        return self._clockwise if offset > 0 else self._anticlockwise
    
    def _position_from_offset(self, offset):
        op = self._move_op(offset)
        position = op(abs(offset))
        return position
        
    def _clockwise(self, distance):
        position = self.position
        remaining_distance = distance
        
        remaining_to_right = len(self.buffer) - position - 1
        if remaining_distance <= remaining_to_right:
            position += remaining_distance
        else:
            position = 0
            remaining_distance -= (remaining_to_right + 1)
            position += remaining_distance % len(self.buffer)
        
        return position

    def _anticlockwise(self, distance):
        position = self.position
        remaining_distance = distance
        
        remaining_to_left = position
        if remaining_distance <= remaining_to_left:
            position -= remaining_distance
        else:
            position = len(self.buffer)
            remaining_distance -= remaining_to_left
            position -= remaining_distance % len(self.buffer)
            
            if position == len(self.buffer):
                position = 0
        
        return position
    
    def __str__(self):
        vals = []
        for pos in range(len(self.buffer)):
            val = self.buffer[pos]
            if pos == self.position:
                vals.append('({})'.format(val))
            else:
                vals.append(str(val))
            
        return ' '.join(vals)     

In [4]:
circle = CircularBuffer([0, 1, 2, 3], 1)
assert CircularBuffer([0, 1, 2, 3], 1).move(7) == 0
assert CircularBuffer([0, 1, 2, 3], 2).move(-7) == 3

In [5]:
circle = CircularBuffer([0])

for x in range(1, 10):
    print(circle.move(2))
    circle.insert(x)
    print(circle)

0
(1) 0
0
(2) 1 0
2
2 1 (3) 0
0
(4) 2 1 3 0
2
4 2 (5) 1 3 0
4
4 2 5 1 (6) 3 0
6
4 2 5 1 6 3 (7) 0
0
(8) 4 2 5 1 6 3 7 0
2
8 4 (9) 2 5 1 6 3 7 0


In [6]:
# Make sure the remove step works
circle_pre_remove = CircularBuffer(
    [0, 16, 8, 17, 4, 18, 9, 19, 2, 20, 10, 21, 5, 22, 11, 1, 12, 6, 13, 3, 14, 7, 15],
    13
)
print(circle_pre_remove)
circle_pre_remove.remove(-7)
circle_pre_remove.move(-6)
print(circle_pre_remove)
circle_pre_remove.move(2)
circle_pre_remove.insert(24)
print(circle_pre_remove)

0 16 8 17 4 18 9 19 2 20 10 21 5 (22) 11 1 12 6 13 3 14 7 15
0 16 8 17 4 18 (19) 2 20 10 21 5 22 11 1 12 6 13 3 14 7 15
0 16 8 17 4 18 19 2 (24) 20 10 21 5 22 11 1 12 6 13 3 14 7 15


In [7]:
# Part 2 - Fast mode is the point at which we'll never hit deletions and new additions
def can_enter_fast_mode(remaining_moves, circle):
    max_distance = 50  # Offset to save working out edges (a little bit extra for luck)...
    max_distance += remaining_moves  # Additions move us one to the right
    max_distance -= remaining_moves // 23 * 7 # Deletions move us 6 back (plus overcounted one above)
    return max_distance < len(circle)

In [8]:
def solve(num_players, num_pieces):
    circle = CircularBuffer([0])
    scores = Counter()
    current_piece = 1
    current_player = 0
    fast_mode = False
    entering_fast_mode = False
    start_time = int(time())
    
    while current_piece <= num_pieces:
        if current_piece % 100000 == 0:
            print('Playing piece {} after {}s...'.format(current_piece, int(time() - start_time)))

        if current_piece % 23 == 0:
            if fast_mode:
                circle.move(16)
                value = circle.value()             
            else:
                value = circle.remove(-7)
                circle.move(-6)

            scores[current_player] += value
            scores[current_player] += current_piece
            
            # Can we go into fast mode ignoring deletions and additions
            # as we'll no longer hit anything to the left
            if not fast_mode and can_enter_fast_mode(num_pieces - current_piece, circle):
                print('Entering fast mode on move {}...'.format(current_piece))
                fast_mode = True
                # We're no longer going to insert or delete items to account
                # For the three we have added before moving back move forward 3
                circle.move(3)   
        else:
            # Nothing to do in fast mode
            if not fast_mode:
                circle.move(2)
                circle.insert(current_piece)
        
        current_piece += 1
        current_player += 1
        if current_player >= num_players:
            current_player = 0
    
    return scores.most_common()[0]   

In [9]:
assert solve(9, 25)[1] == 32

#10 players; last marble is worth 1618 points: high score is 8317
#13 players; last marble is worth 7999 points: high score is 146373
#17 players; last marble is worth 1104 points: high score is 2764
#21 players; last marble is worth 6111 points: high score is 54718
#30 players; last marble is worth 5807 points: high score is 37305

assert solve(10, 1618)[1] == 8317
assert solve(13, 7999)[1] == 146373
assert solve(17, 1104)[1] == 2764
assert solve(21, 6111)[1] == 54718
assert solve(30, 5807)[1] == 37305

Entering fast mode on move 736...
Entering fast mode on move 3496...
Entering fast mode on move 529...
Entering fast mode on move 2691...
Entering fast mode on move 2553...


In [10]:
solve(10, 1618)

Entering fast mode on move 736...


(9, 8317)

In [11]:
%%time

# 448 players; last marble is worth 71628 points
print(solve(448, 71628))

Entering fast mode on move 31027...
(170, 394486)
CPU times: user 268 ms, sys: 6.99 ms, total: 275 ms
Wall time: 286 ms


### Part 2

Amused by the speed of your answer, the Elves are curious:

What would the new winning Elf's score be if the number of the last marble were 100 times larger?

In [13]:
# Test fast mode behaviour
# After a point we will never hit addition and deletions to the buffer
# so let's work out equivalent behaviour without them

mutating_circle = CircularBuffer(list(range(1000)))
static_circle = CircularBuffer(list(range(1000)))

for x in range(1, 300):
    if x % 23 == 0:
        if x == 23:
            # Hit three extra values that I did actually insert
            # Move forward to compensate
            static_circle.move(19)
        else:
            static_circle.move(16)

        removed = mutating_circle.remove(-7)
        mutating_circle.move(-6)
        
        print('FAST:', static_circle.value())
        print('SLOW:', removed)
    else:
        mutating_circle.move(2)
        mutating_circle.insert('X')

FAST: 19
SLOW: 19
FAST: 35
SLOW: 35
FAST: 51
SLOW: 51
FAST: 67
SLOW: 67
FAST: 83
SLOW: 83
FAST: 99
SLOW: 99
FAST: 115
SLOW: 115
FAST: 131
SLOW: 131
FAST: 147
SLOW: 147
FAST: 163
SLOW: 163
FAST: 179
SLOW: 179
FAST: 195
SLOW: 195
FAST: 211
SLOW: 211


In [14]:
%%time

print(solve(448, 71628 * 100))

Playing piece 100000 after 3s...
Playing piece 200000 after 8s...
Playing piece 300000 after 16s...
Playing piece 400000 after 20s...
Playing piece 500000 after 46s...
Playing piece 600000 after 67s...
Playing piece 700000 after 84s...
Playing piece 800000 after 94s...
Playing piece 900000 after 98s...
Playing piece 1000000 after 140s...
Playing piece 1100000 after 203s...
Playing piece 1200000 after 266s...
Playing piece 1300000 after 316s...
Playing piece 1400000 after 363s...
Playing piece 1500000 after 400s...
Playing piece 1600000 after 429s...
Playing piece 1700000 after 455s...
Playing piece 1800000 after 478s...
Playing piece 1900000 after 492s...
Playing piece 2000000 after 503s...
Playing piece 2100000 after 509s...
Playing piece 2200000 after 517s...
Playing piece 2300000 after 692s...
Playing piece 2400000 after 897s...
Playing piece 2500000 after 1026s...
Playing piece 2600000 after 1150s...
Playing piece 2700000 after 1269s...
Playing piece 2800000 after 1383s...
Playing 